# simran4@wisc.edu, rgundavarapu@wisc.edu

# PART 1: Data Setup

In [1]:
from pyspark.sql import SparkSession
spark = (SparkSession.builder.appName("cs544")
         .master("spark://main:7077")
         .config("spark.executor.memory", "512M")
         .config("spark.sql.warehouse.dir", "hdfs://main:9000/user/hive/warehouse")
         .enableHiveSupport()
         .getOrCreate())

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/03/31 21:33:24 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
!curl https://pages.cs.wisc.edu/~harter/cs639/data/hdma-wi-2021.zip > wi-2021.zip
!curl https://pages.cs.wisc.edu/~harter/cs639/data/arid2017_to_lei_xref_csv.zip > arid-2017.zip
!curl https://pages.cs.wisc.edu/~harter/cs639/data/code_sheets.zip > code_sheets.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 20.4M  100 20.4M    0     0  8803k      0  0:00:02  0:00:02 --:--:-- 8804k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  141k  100  141k    0     0   417k      0 --:--:-- --:--:-- --:--:--  418k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  779k  100  779k    0     0  1811k      0 --:--:-- --:--:-- --:--:-- 1808k


In [3]:
import zipfile as zf
import glob
 
zips = [zf.ZipFile('wi-2021.zip'),zf.ZipFile('arid-2017.zip'),zf.ZipFile('code_sheets.zip')]

for zipfiles in zips:
    zipfiles.extractall()
    
csv_dict = {}
path = "./*.csv"
for file_name in glob.glob(path):
    file_name = str(file_name[2:])
    !hdfs dfs -cp {file_name} hdfs://main:9000/{file_name}

In [4]:
!hdfs dfs -du -h hdfs://main:9000/

328      984      hdfs://main:9000/action_taken.csv
317      951      hdfs://main:9000/agency.csv
521.0 K  1.5 M    hdfs://main:9000/arid2017_to_lei_xref_csv.csv
311.6 K  934.8 K  hdfs://main:9000/counties.csv
237      711      hdfs://main:9000/denial_reason.csv
109      327      hdfs://main:9000/edit_status.csv
180      540      hdfs://main:9000/ethnicity.csv
166.8 M  500.5 M  hdfs://main:9000/hdma-wi-2021.csv
41       123      hdfs://main:9000/hoepa.csv
114      342      hdfs://main:9000/lien_status.csv
65       195      hdfs://main:9000/loan_purpose.csv
79       237      hdfs://main:9000/loan_type.csv
129.6 K  388.7 K  hdfs://main:9000/msamd.csv
122      366      hdfs://main:9000/owner_occupancy.csv
92       276      hdfs://main:9000/preapproval.csv
127      381      hdfs://main:9000/property_type.csv
387      1.1 K    hdfs://main:9000/purchaser_type.csv
252      756      hdfs://main:9000/race.csv
144      432      hdfs://main:9000/sex.csv
955      2.8 K    hdfs://main:9000/states.c

In [4]:
table_list = ["hdma-wi-2021","arid2017_to_lei_xref_csv", "ethnicity", "race", "sex", "states", 
              "counties","tracts", "action_taken", "denial_reason", "loan_type", "loan_purpose", 
              "preapproval", "property_type"]

for zipfiles in zips:
    for csv_files in zipfiles.namelist():
        if(csv_files[:-4] in table_list):
            #print(csv_files)
            df = (spark.read.format("csv").option("header", True).option("inferSchema", True)
                  .load("hdfs://main:9000/" + csv_files))
            if(csv_files == 'hdma-wi-2021.csv'):
                df.write.bucketBy(8,"county_code").saveAsTable('loans', mode='overwrite')
            elif(csv_files == 'arid2017_to_lei_xref_csv.csv'):                                  
                df.write.saveAsTable('banks', mode='overwrite')      
            else:
                df.createOrReplaceTempView(csv_files[:-4])
            #print(csv_files)

23/03/31 00:54:16 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
23/03/31 00:54:16 WARN HiveConf: HiveConf of name hive.stats.retries.wait does not exist
23/03/31 00:54:21 WARN ObjectStore: Version information not found in metastore. hive.metastore.schema.verification is not enabled so recording the schema version 2.3.0
23/03/31 00:54:21 WARN ObjectStore: setMetaStoreSchemaVersion called but recording version is disabled: version = 2.3.0, comment = Set by MetaStore UNKNOWN@172.18.0.3
23/03/31 00:54:22 WARN ObjectStore: Failed to get database global_temp, returning NoSuchObjectException
23/03/31 00:54:22 WARN HadoopFSUtils: The directory hdfs://main:9000/user/hive/warehouse/loans was not found. Was it deleted very recently?
23/03/31 00:54:23 WARN FileUtils: File does not exist: hdfs://main:9000/user/hive/warehouse/loans; Force to delete it.
23/03/31 00:54:23 ERROR FileUtils: Failed to delete hdfs://main:9000/user/hive/warehouse/loans
23/03/31 00:54:23 WARN packag

# Q1

In [5]:
spark.sql("SHOW TABLES").show()

+---------+-------------+-----------+
|namespace|    tableName|isTemporary|
+---------+-------------+-----------+
|  default|        banks|      false|
|  default|        loans|      false|
|         | action_taken|       true|
|         |     counties|       true|
|         |denial_reason|       true|
|         |    ethnicity|       true|
|         | loan_purpose|       true|
|         |    loan_type|       true|
|         |  preapproval|       true|
|         |property_type|       true|
|         |         race|       true|
|         |          sex|       true|
|         |       states|       true|
|         |       tracts|       true|
+---------+-------------+-----------+



# PART 2: Filter and Join

# Q2

In [6]:
spark.sql("""
SELECT COUNT(*)
FILTER(
WHERE LOWER(respondent_name) LIKE "%first%") AS firsts
FROM banks
""").toPandas()

,firsts
0,525


In [7]:
spark.sql("""
SELECT *
FROM banks
WHERE LOWER(respondent_name) LIKE "%second%"
""").toPandas()

,respondent_name,arid_2017,lei_2018,lei_2019,lei_2020
0,Old Second National Bank,14596,B94HV8SXGDD3J5XESU08,B94HV8SXGDD3J5XESU08,B94HV8SXGDD3J5XESU08


# Q3

In [8]:
results = spark.sql(
    """
    SELECT COUNT(*) 
    FROM banks
    INNER JOIN loans ON loans.lei = banks.lei_2020
    WHERE banks.respondent_name = 'AMERICAN FINANCIAL NETWORK, INC.'
    """)
results.toPandas()

## FAVOURITE BANK IS AMERICAN FINANCIAL NETWORK, INC.

,count(1)
0,564


# Q4

In [9]:
results.explain('formatted')

== Physical Plan ==
AdaptiveSparkPlan (25)
+- == Final Plan ==
   * HashAggregate (15)
   +- ShuffleQueryStage (14)
      +- Exchange (13)
         +- * HashAggregate (12)
            +- * Project (11)
               +- * BroadcastHashJoin Inner BuildLeft (10)
                  :- BroadcastQueryStage (6)
                  :  +- BroadcastExchange (5)
                  :     +- * Project (4)
                  :        +- * Filter (3)
                  :           +- * ColumnarToRow (2)
                  :              +- Scan parquet default.banks (1)
                  +- * Filter (9)
                     +- * ColumnarToRow (8)
                        +- Scan parquet default.loans (7)
+- == Initial Plan ==
   HashAggregate (24)
   +- Exchange (23)
      +- HashAggregate (22)
         +- Project (21)
            +- BroadcastHashJoin Inner BuildLeft (20)
               :- BroadcastExchange (18)
               :  +- Project (17)
               :     +- Filter (16)
               :        +-

1. Which table is sent to every executor via a BroadcastExchange operation?
* loans
2. On which tables is "is not null" filtering added by the optimizer?
* banks
* loans
3. Which Functions do the HashAggregates use?
* partial_count(1)
* count(1)

# Q5

In [ ]:
# spark.sql(
#     """
#     SELECT activity_year
#     FROM loans
#     """).toPandas()

In [131]:
# spark.sql(
#     """
#     SELECT loans.census_tract, loans.derived_dwelling_category
#     FROM banks
#     LEFT JOIN loans ON 
#     loans.lei = banks.lei_2020
    
#     WHERE banks.respondent_name = 'AMERICAN FINANCIAL NETWORK, INC.' 
#     """).toPandas()

#INNER JOIN loans ON loans.lei = banks.lei_2020

,census_tract,derived_dwelling_category
0,55101002401,Single Family (1-4 Units):Site-Built
1,55101001602,Single Family (1-4 Units):Site-Built
2,55101002002,Single Family (1-4 Units):Site-Built
3,55101002701,Single Family (1-4 Units):Site-Built
4,55101002002,Single Family (1-4 Units):Site-Built
...,...,...
559,55079009700,Single Family (1-4 Units):Site-Built
560,55079021600,Single Family (1-4 Units):Site-Built
561,55079140202,Single Family (1-4 Units):Site-Built
562,55079060102,Single Family (1-4 Units):Site-Built


In [71]:
# spark.sql(
#     """
#     SELECT *
#     FROM action_taken
#     """).toPandas()

,id,action_taken
0,1,Loan originated
1,2,Application approved but not accepted
2,3,Application denied by financial institution
3,4,Application withdrawn by applicant
4,5,File closed for incompleteness
5,6,Loan purchased by the institution
6,7,Preapproval request denied by financial instit...
7,8,Preapproval request approved but not accepted


In [66]:
# spark.sql(
#     """
#     SELECT *
#     FROM action_taken
#     WHERE action_taken.action_taken = "Loan originated"
#     LIMIT 5
#     """).toPandas()

,id,action_taken
0,1,Loan originated


In [64]:
# spark.sql(
#     """
#     SELECT *
#     FROM banks
#     LIMIT 5
#     """).toPandas()

,respondent_name,arid_2017,lei_2018,lei_2019,lei_2020
0,First National Bank,110004,5493003EW6T31TGECO83,5493003EW6T31TGECO83,5493003EW6T31TGECO83
1,"First Mid Bank & Trust, National Association",110045,549300XOTES5TCS8T794,549300XOTES5TCS8T794,549300XOTES5TCS8T794
2,"First Hope Bank, A National Banking Association",110118,5493003XLOX5FDT9R120,5493003XLOX5FDT9R120,5493003XLOX5FDT9R120
3,First National Bank of Waterloo,110180,5493002GOGIUXB84JB89,5493002GOGIUXB84JB89,5493002GOGIUXB84JB89
4,Cumberland Valley National Bank & Trust Company,110254,549300OULGV8ZW0JV320,549300OULGV8ZW0JV320,549300OULGV8ZW0JV320


In [68]:
# results.toPandas()

,count(1)
0,564


In [132]:
spark.sql(
    """
    SELECT *
    FROM loans
    LIMIT 10
    """).toPandas()

,activity_year,lei,derived_msa-md,state_code,county_code,census_tract,conforming_loan_limit,derived_loan_product_type,derived_dwelling_category,derived_ethnicity,...,denial_reason-2,denial_reason-3,denial_reason-4,tract_population,tract_minority_population_percent,ffiec_msa_md_median_family_income,tract_to_msa_income_percentage,tract_owner_occupied_units,tract_one_to_four_family_homes,tract_median_age_of_housing_units
0,2021,5493008NWHQT1R22C024,33340,WI,55079,55079101600,C,Conventional:First Lien,Single Family (1-4 Units):Site-Built,Not Hispanic or Latino,...,NaN,NaN,NaN,4800,27.63,84400,88,1045,1823,66
1,2021,549300HZMPZC336WP721,31540,WI,55025,55025012600,C,Conventional:First Lien,Single Family (1-4 Units):Site-Built,Not Hispanic or Latino,...,NaN,NaN,NaN,4606,4.36,99000,100,1415,1824,44
2,2021,5493008NWHQT1R22C024,33340,WI,55079,55079060102,C,Conventional:First Lien,Single Family (1-4 Units):Site-Built,Not Hispanic or Latino,...,NaN,NaN,NaN,3205,28.67,84400,139,1148,1001,43
3,2021,549300HZMPZC336WP721,31540,WI,55025,55025012400,C,Conventional:First Lien,Single Family (1-4 Units):Site-Built,Not Hispanic or Latino,...,NaN,NaN,NaN,4940,3.64,99000,127,1714,1895,30
4,2021,5493008NWHQT1R22C024,33340,WI,55079,55079140202,C,FHA:First Lien,Single Family (1-4 Units):Site-Built,Ethnicity Not Available,...,NaN,NaN,NaN,6135,7.68,84400,112,1563,2096,48
5,2021,549300HZMPZC336WP721,31540,WI,55025,55025011506,C,Conventional:First Lien,Single Family (1-4 Units):Site-Built,Not Hispanic or Latino,...,NaN,NaN,NaN,4939,18.97,99000,84,755,1280,21
6,2021,5493008NWHQT1R22C024,33340,WI,55079,55079021600,C,FHA:First Lien,Single Family (1-4 Units):Site-Built,Ethnicity Not Available,...,NaN,NaN,NaN,4580,30.13,84400,69,1198,1435,46
7,2021,549300HZMPZC336WP721,33340,WI,55079,55079150301,C,Conventional:First Lien,Single Family (1-4 Units):Site-Built,Ethnicity Not Available,...,NaN,NaN,NaN,6211,11.16,84400,124,1942,1931,40
8,2021,5493008NWHQT1R22C024,33340,WI,55079,55079009700,C,FHA:First Lien,Single Family (1-4 Units):Site-Built,Ethnicity Not Available,...,NaN,NaN,NaN,1745,94.56,84400,81,190,520,76
9,2021,549300HZMPZC336WP721,31540,WI,55025,55025000700,C,Conventional:First Lien,Single Family (1-4 Units):Site-Built,Not Hispanic or Latino,...,NaN,NaN,NaN,3221,14.13,99000,155,1078,1202,62


In [89]:
# spark.sql(
#     """
#     SHOW TABLES
#     """).show()

+---------+-------------+-----------+
|namespace|    tableName|isTemporary|
+---------+-------------+-----------+
|  default|        banks|      false|
|  default|        loans|      false|
|         | action_taken|       true|
|         |     counties|       true|
|         |denial_reason|       true|
|         |    ethnicity|       true|
|         | loan_purpose|       true|
|         |    loan_type|       true|
|         |  preapproval|       true|
|         |property_type|       true|
|         |         race|       true|
|         |          sex|       true|
|         |       states|       true|
|         |       tracts|       true|
+---------+-------------+-----------+



In [142]:
# spark.sql(
#     """
#     SELECT DISTINCT *
#     FROM counties
#     """).toPandas()

,AREA,PERIMETER,CO99_D00_,CO99_D00_I,STATE,COUNTY,NAME,LSAD,LSAD_TRANS
0,0.71343,4.73735,185.0,184.0,30,99,Teton,6,County
1,1.34881,7.77343,384.0,383.0,16,59,Lemhi,6,County
2,1.22568,5.00577,505.0,504.0,56,39,Teton,6,County
3,0.20002,2.13064,553.0,552.0,55,1,Adams,6,County
4,0.20654,1.87734,792.0,791.0,55,105,Rock,6,County
...,...,...,...,...,...,...,...,...,...
3484,0.23945,2.55503,3235.0,3234.0,48,245,Jefferson,6,County
3485,0.00168,0.19176,3379.0,3378.0,12,87,Monroe,6,County
3486,0.00005,0.02748,3427.0,3426.0,72,53,Fajardo,13,Municipio
3487,0.00651,0.48548,3428.0,3427.0,72,53,Fajardo,13,Municipio


In [105]:
spark.sql(
    """
    SELECT *
    FROM loan_purpose
    LIMIT 5
    """).toPandas()

,id,loan_purpose
0,1,Home purchase
1,2,Home improvement
2,3,Refinancing


In [134]:
# spark.sql(
#     """
#     SELECT *
#     FROM loan_type
#     LIMIT 5
#     """).toPandas()

,id,loan_type
0,1,Conventional
1,2,FHA-insured
2,3,VA-guaranteed
3,4,FSA/RHS-guaranteed


In [ ]:
# banks.toPandas()

In [148]:
loans = spark.table('loans')
banks = spark.table('banks')
counties = spark.table('counties').distinct()
lb = banks.join(loans, on=(loans["lei"] == banks["lei_2020"]), how="left")
results=lb.where("respondent_name == 'AMERICAN FINANCIAL NETWORK, INC.'").count()


In [149]:
# results

564